<img src="images/crepmaker.png" width="600" align="left"/>

In [3]:
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # This is added back by InteractiveShellApp.init_path()


device(type='cuda')

## Step one: copping some clean kicks
I scraped images off [Flight Club](https://flightclub.com/) to use as training data. They're a sneaker retailer in New York that stock tons of sneakers but more importantly: they have great pics of their sneakers on their online store! So I obviously do not own any of the images, and the copyright belongs to Flight Club. Please don't sue me✌🏽.

The dataset consists of these photos, without any labels or metadata beyond the image data.

In [4]:
plt.figure(figsize=(18,7))
selected_kicks = [33, 59, 398, 104, 151]

for i in np.arange(1, 6):
    plt.subplot(1, 5, i)
    plt.axis('off')
    img = plt.imread('snkrs/0/{}.jpg'.format(selected_kicks[i-1]))
    plt.imshow(img)

In [75]:
IMG_WIDTH = 750
IMG_HEIGHT = 536
HIDDEN_DIM = 402000
LATENT_DIM = 10

img_transforms = transforms.Compose([
        transforms.CenterCrop((IMG_HEIGHT, IMG_WIDTH)),
        transforms.transforms.ToTensor(),
])

sneaker_dataset = datasets.ImageFolder(
        root='./snkrs',
        transform=img_transforms
)

sneaker_dataloader = DataLoader(
        sneaker_dataset,
        batch_size=1,
        num_workers=2,
        shuffle=True
)

## Step two: VAE

In [79]:
class SneakerVAE(nn.Module):
    def __init__(self):
        super(SneakerVAE, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(HIDDEN_DIM, LATENT_DIM)
        self.fc2 = nn.Linear(HIDDEN_DIM, LATENT_DIM)
        self.fc3 = nn.Linear(LATENT_DIM, HIDDEN_DIM)
        self.convT1 = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1)
        self.convT2 = nn.ConvTranspose2d(64, 64, kernel_size=3, padding=1)
        self.convT3 = nn.ConvTranspose2d(64, 3, kernel_size=1)

    def encode(self, x):
        # print(x.shape)
        x = F.relu(self.conv1(x))
        # print(x.shape)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        # print(x.shape)
        batch_dim = x.shape[0]
        x = x.reshape([batch_dim, 64, -1])
        return self.fc1(x), self.fc2(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        z = F.relu(self.convT1(z))
        z = F.relu(self.convT2(z))
        z = self.convT3(z)
        return torch.sigmoid(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        z = self.fc3(z)
        batch_dim = z.shape[0]
        z = z.reshape([batch_dim, 64, IMG_HEIGHT, IMG_WIDTH])
        return self.decode(z), mu, logvar
    
    
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    # KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    KLD = ((mu**2 + logvar.exp() - 1 - logvar) / 2).mean()

    if torch.isnan(BCE+KLD):
        print('BCE: {}'.format(BCE))
        print('KLD: {}'.format(KLD))
        print()
    return BCE + KLD

In [78]:
sneaker_vae = SneakerVAE().to(device)
sneaker_vae(data);

In [125]:
# sneaker_vae = SneakerVAE().to(device)
optimizer = optim.Adam(sneaker_vae.parameters(), lr=1e-4)

sneaker_vae.train()

for epoch in range(7,11):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(sneaker_dataloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = sneaker_vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(sneaker_dataset),
                100. * batch_idx / len(sneaker_dataloader),
                loss.item() ))

    torch.save(sneaker_vae.state_dict, 'models/{}.pth'.format(epoch))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(sneaker_dataset)))

Train Epoch: 7 [0/939 (0%)]	Loss: 261803.609375
Train Epoch: 7 [50/939 (5%)]	Loss: 318916.937500
Train Epoch: 7 [100/939 (11%)]	Loss: 239485.000000
Train Epoch: 7 [150/939 (16%)]	Loss: 332751.062500
Train Epoch: 7 [200/939 (21%)]	Loss: 211304.593750
Train Epoch: 7 [250/939 (27%)]	Loss: 289700.343750
Train Epoch: 7 [300/939 (32%)]	Loss: 246511.250000
Train Epoch: 7 [350/939 (37%)]	Loss: 317719.875000
Train Epoch: 7 [400/939 (43%)]	Loss: 348004.531250
Train Epoch: 7 [450/939 (48%)]	Loss: 260416.562500
Train Epoch: 7 [500/939 (53%)]	Loss: 257694.281250
Train Epoch: 7 [550/939 (59%)]	Loss: 277703.906250
Train Epoch: 7 [600/939 (64%)]	Loss: 172564.468750
Train Epoch: 7 [650/939 (69%)]	Loss: 218523.828125
Train Epoch: 7 [700/939 (75%)]	Loss: 232745.828125
Train Epoch: 7 [750/939 (80%)]	Loss: 205168.734375
Train Epoch: 7 [800/939 (85%)]	Loss: 152354.578125
Train Epoch: 7 [850/939 (91%)]	Loss: 329060.812500
Train Epoch: 7 [900/939 (96%)]	Loss: 230293.953125
====> Epoch: 7 Average loss: 262283.

In [12]:
def test_image(idx):
    plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.title('Actual')
    plt.imshow(sneaker_dataset[idx][0].permute(1,2,0))
    plt.subplot(1, 2, 2)
    plt.axis('off')
    plt.title('Reconstructed')
    plt.imshow( sneaker_vae(sneaker_dataset[idx][0].unsqueeze(0).to(device))[0].detach().cpu().squeeze().permute(1,2,0).numpy() )

In [185]:
test_image(8)

In [149]:
torch.isnan(torch.tensor(0))

tensor(False)